In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys


sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

from llm_lexical_simplifier import LLMLexicalSimplifier
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
from benchmark_suite import BenchmarkSuite
from language import Language
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BertForMaskedLM
from tqdm import tqdm

Mounted at /content/drive


# LLM based

Install this and the restart the session. This is a workaround for a known bug in flash-attn. Do not re-run this cell after restarting the session.

In [2]:
#!pip install accelerate flash-attn

In [2]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    #device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    # Flash attention is only supported starting Ampere architecture
    # So we cant use it on free-tier GPUs
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

# Define PAD Token = EOS Token
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [4]:
de_messages = [
    {"role": "user", "content": "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Erleichterung erhalten. Die vereinfachte Version des vorigen Satzes ist: Der Maschinenbau hat in Europa durch die Bildung der EU eine starke [MASK] erhalten. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Vereinfachung", "Entspannung", "Begünstigung", "Unterstützung", "Rückenwind,  "Förderung", "Rückhalt", "Aufwind", "Antrieb", "Erleichterung"]'},
    {"role": "user", "content": "Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen Sanktus signalisiert. Die vereinfachte Version des vorigen Satzes ist: Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen [MASK] signalisiert. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Zustimmung", "Einverständnis", "Okay", "Segen", "Billigung, Bestätigung, Bejahung, Zuspruch, Akzeptanz, Befürwortung"]'},
]
en_messages = [
    {"role": "user", "content": "The president’s relatives were also said to be critical of the way his closest advisers had prepared him for the debate. The simplified version of the previous sentence is: The president’s relatives were also said to be critical of the way his closest [MASK] had prepared him for the debate. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["staff", "team", "aides", "consultants", "assistants", "strategists", "supporters", "handlers", "counselors", "staffers"]'},
    {"role": "user", "content": "The intermediate sprint will come 94km or so into the stage. The simplified version of the previous sentence is: The intermediate sprint will come 94km or so into the [MASK]. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segment", "part", "section", "phase", "leg", "interval", "course", "route", "portion", "strech"]'},
]

llm_ls = LLMLexicalSimplifier(model=model, tokenizer=tokenizer,
                              pattern='{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Ten results as ordered, AST parsable Python list:',
                              exemplars=en_messages, mask_token='[MASK]', generation_args=generation_args)

Using cuda for model inference.
Using mask token: "[MASK]".


In [5]:
llm_output = llm_ls.generate_substitutions_for('heikel', 'Die politische Lage in Syrien ist heikel.')
llm_output

['heikel',
 'kompliziert',
 'schwierig',
 'risikabel',
 'unstabil',
 'bedrohlich',
 'gefährlich',
 'kränklich',
 'problematisch',
 'schwierig']

In [14]:
suite = BenchmarkSuite(llm_ls, {
        Language.EN: {'pattern': '{original_sentence}. The simplified version of the previous sentence is: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                      'exemplars': en_messages}})
suite.run()

NameError: name 'llm_ls' is not defined

#BERT based

In [8]:
bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
bert_model = BertForMaskedLM.from_pretrained("google-bert/bert-base-cased")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
bert_ls = SimpleBertLexicalSimplifier(bert_model, bert_tokenizer, '{original_sentence}. The simplified version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', None, '[MASK]')

No exemplars provided, using zero-shot mode.
Using mask token: "[MASK]".


In [17]:
suite = BenchmarkSuite(bert_ls, {Language.EN: {'pattern': '{original_sentence}. The simpler version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:',
                                               'exemplars': None},
                                 Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Die zehn besten Resultate:',
                                               'exemplars': None}})
suite.run()

Benchmarking model on EN ...
Benchmarking model on BenchLSDataProvider...


Benchmarking:  42%|████▏     | 386/929 [00:09<00:13, 38.89it/s]


KeyboardInterrupt: 